In [37]:
import requests
import smtplib
from datetime import datetime, timedelta
import re

In [3]:
# https://www.alphavantage.co/documentation/
# https://newsapi.org/docs/client-libraries/python

API_key = ""
API_news_key = ""

CRYPTO_ENDPOINT = "https://www.alphavantage.co/query"
NEWS_ENDPOINT = "https://newsapi.org/v2/everything"

CRYPTOCURRENCY = "BTC"
CRYPTO_NAME = "Bitcoin"
CURRENCY = "USD"
# EUR, JPY, HKD, GBP

# API Parameters
btc_parameters = {
    "function":"DIGITAL_CURRENCY_DAILY",
    "symbol":CRYPTOCURRENCY,
    "market": CURRENCY,
    "apikey":API_key
}

response = requests.get(url=CRYPTO_ENDPOINT, params=btc_parameters)
response.raise_for_status()
json_data = response.json()

time_series_daily = json_data['Time Series (Digital Currency Daily)']

In [4]:
# https://stackoverflow.com/questions/1712116/formatting-yesterdays-date-in-python
today = datetime.today()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
day_before = today - timedelta(days=2)
day_before = day_before.strftime("%Y-%m-%d")

In [5]:
# yesterday's and day's before closing points
# we need to take this key '4b. close (USD)'
yesterday_close = float(time_series_daily[yesterday]['4b. close (USD)'])
day_before_close = float(time_series_daily[day_before]['4b. close (USD)'])

43451.13 45832.01


In [13]:
# positive difference --> use abs() to get the absolute value of the result
# if pos_difference is positive create a string that says "stock up to", else "stock down to"
# pos_difference has to have a sign --> remove abs()
pos_difference = yesterday_close - day_before_close

if pos_difference > 0:
    trend = "up to "
else:
    trend = "down to "

print(trend)

down to 


In [7]:
# calculate the percentage difference
# https://www.calculatorsoup.com/calculators/algebra/percent-difference-calculator.php
diff_percent = round((pos_difference / yesterday_close) * 100,2)

# here we have to use abs() in order to get rid of the sign
diff_percent = abs(diff_percent)
diff_percent

5.48

In [56]:
# our threshold is 5 and if diff_percentage is greater than 5 then we get the news
# for testing purposes we use lower threshold than diff_percentage

news_parameters = {
    "apiKey":API_news_key,
    "qInTitle": CRYPTO_NAME
}

if diff_percent > 5:
    # get the news
    news = requests.get(url=NEWS_ENDPOINT, params=news_parameters)
    news.raise_for_status()
    # grab top three articles
    articles = news.json()['articles'][:3]

In [52]:
articles

[{'source': {'id': None, 'name': 'New York Times'},
  'author': 'Corey Kilgannon',
  'title': 'Why New York State Is Experiencing a Bitcoin Boom',
  'description': 'Cryptocurrency miners are flocking to New York’s faded industrial towns, prompting concern over the environmental impact of huge computer farms.',
  'url': 'https://www.nytimes.com/2021/12/05/nyregion/bitcoin-mining-upstate-new-york.html',
  'urlToImage': 'https://static01.nyt.com/images/2021/11/25/nyregion/00nybitcoin5/00nybitcoin5-facebookJumbo.jpg',
  'publishedAt': '2021-12-06T00:42:28Z',
  'content': 'The plant opening northeast of Niagara Falls this month, in Somerset, N.Y., is part of a $550 million project by Terawulf, a Bitcoin mining company. The project also includes a proposed 150-megawatt … [+1514 chars]'},
 {'source': {'id': 'bbc-news', 'name': 'BBC News'},
  'author': 'https://www.facebook.com/bbcnews',
  'title': "Indian PM Modi's Twitter hacked with bitcoin tweet",
  'description': "The Indian prime ministe

In [60]:
# format articles by using a list comprehension
f_articles = [f"{CRYPTO_NAME}: {trend}{diff_percent}%\nHeadline: {article['title']}\nBrief: {article['description']}" for article in articles]

# UnicodeEncodeError: 'ascii' codec can't encode character '\u2019' in position 229: ordinal not in range(128)
f_articles = [re.sub(r"[“”’'-]", '',article) for article in f_articles]
print(f_articles)

['Bitcoin: down to 5.48%\nHeadline: Why New York State Is Experiencing a Bitcoin Boom\nBrief: Cryptocurrency miners are flocking to New Yorks faded industrial towns, prompting concern over the environmental impact of huge computer farms.', 'Bitcoin: down to 5.48%\nHeadline: Indian PM Modis Twitter hacked with bitcoin tweet\nBrief: The Indian prime ministers account had a message stating that bitcoin would be distributed to citizens.', 'Bitcoin: down to 5.48%\nHeadline: Selfproclaimed bitcoin inventor largely prevails in $54 billion bitcoin trial  Reuters.com\nBrief: A computer scientist who has claimed to be the inventor of bitcoin largely prevailed on Monday in a Florida jury trial over whether the estate of a former business partner deserved half of a bitcoin cache worth now about $54 billion.']


In [61]:
# https://stackoverflow.com/questions/9942594/unicodeencodeerror-ascii-codec-cant-encode-character-u-xa0-in-position-20
# https://gankrin.org/fix-unicodeencodeerror-ascii-codec-cant-encode-character/

my_email = ""
pw = ""
target = ""

# send emails
for article in f_articles:
    with smtplib.SMTP("smtp.gmail.com") as connection:
        connection.starttls()
        connection.login(user=my_email, password=pw)
        connection.sendmail(from_addr=my_email, to_addrs=target, msg=f"Subject:Bitcoin News \n\n{article}")